In [1]:
import os
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from bachfile import BachDataset
import torch.optim as optim
import time
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()
import copy

2023-04-14 21:00:09.730614: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 21:00:11.667597: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-14 21:00:11.668235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-14 21:00:11.668243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Resize([512, 384])])

batch_size = 32

train_dataset = BachDataset(path='/home/ardhendu/Desktop/bach_dataset_classification/data-train.csv',transforms=transform )
val_dataset = BachDataset(path='/home/ardhendu/Desktop/bach_dataset_classification/data-test.csv' ,transforms=transform)

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=False)
testloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=False)


device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [3]:
# load the model in a new file for inference
resnet18_new = torchvision.models.resnet18()
# note that we need to create exactly the same backbone in order to load the weights
backbone_new = nn.Sequential(*list(resnet18_new.children())[:-1])
ckpt = torch.load('/home/ardhendu/Desktop/model.pth')
backbone_new.load_state_dict(ckpt['resnet18_parameters'])
for param in backbone_new.parameters():
    param.requires_grad = False
#last_conv_channels = list(resnet18_new.children())[-1].in_features
#print(last_conv_channels)

In [4]:
#num_ftrs =resnet18_new.avgpool.in_features
#backbone_new.fc1 = nn.Linear(1,16384)
#backbone_new.fc2 = nn.Linear(16384,num_ftrs)
#backbone_new.conv = nn.Conv2d(last_conv_channels, 512, 1)
#backbone_new.avgpool1 = nn.AdaptiveAvgPool2d(1)
backbone_new.fl = nn.Flatten()
backbone_new.fc = nn.Linear(512,4)
backbone_new = backbone_new.to(device)
scaler = torch.cuda.amp.GradScaler()
criterion = nn.CrossEntropyLoss()

In [5]:
optimizer = optim.AdamW(backbone_new.parameters(), lr=3e-4, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)

In [6]:
best_acc = 0.0
train_accuracy=[]
train_loss=[]
val_accuracy=[]
val_loss=[]
for epoch in range(500):  # loop over the dataset multiple times
    t0 = time.time()
    
    running_corrects = 0
    running_loss = 0.0

    for data in tqdm(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        #print(labels)
        optimizer.zero_grad()
        outputs = backbone_new(inputs)
        #print(f' out :{outputs.shape} and label :{labels.shape} and img :{inputs.shape}')
        s_out= F.softmax(outputs)
        _, preds = torch.max(outputs, 1)
        with torch.cuda.amp.autocast():
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).cpu()    
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset)
    train_accuracy.append(epoch_acc)
    train_loss.append(epoch_loss)
    writer.add_scalar("Loss/train", epoch_loss, epoch+1)
    writer.add_scalar("Acc/train", epoch_acc, epoch+1)    
    running_test_loss=0
    running_test_correct=0


    for data in tqdm(testloader):
        images, labels = data[0].to(device), data[1].to(device)
        with torch.no_grad():
            outputs = backbone_new(images)
            s_out= F.softmax(outputs)
        _, predicted = torch.max(outputs.data, 1)
        test_loss= criterion(outputs,labels)
        running_test_loss=test_loss.item()*images.size(0)
        running_test_correct+= torch.sum(predicted == labels.data).cpu()
       
    epoch_val_loss = running_test_loss / len(val_dataset)
    epoch_val_acc = running_test_correct.double() / len(val_dataset)
    val_accuracy.append(epoch_val_acc)
    val_loss.append(epoch_val_loss)
    writer.add_scalar("Loss/Val", epoch_val_loss, epoch+1)
    writer.add_scalar("Acc/Val", epoch_val_acc, epoch+1)
    print(f"Epoch : {epoch+1} - Train_loss : {epoch_loss:.4f} - Train_Acc: {epoch_acc:.4f} - Val_loss : {epoch_val_loss:.4f} - Val_Acc: {epoch_val_acc:.4f}  - Time: {time.time() - t0}\n")
    if epoch_val_acc > best_acc:
        best_acc = epoch_val_acc
        best_model_wts = copy.deepcopy(backbone_new.state_dict())
        
#torch.save(best_model_wts, PATH)
print('Finished Training')


  0%|                                                     | 0/9 [00:00<?, ?it/s]/tmp/ipykernel_98917/1319659284.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  s_out= F.softmax(outputs)
  0%|                                                     | 0/4 [00:00<?, ?it/s]/tmp/ipykernel_98917/1319659284.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  s_out= F.softmax(outputs)
100%|█████████████████████████████████████████████| 4/4 [00:59<00:00, 14.91s/it]


Epoch : 1 - Train_loss : 1.3880 - Train_Acc: 0.2000 - Val_loss : 0.2735 - Val_Acc: 0.2667  - Time: 208.23814868927002



100%|█████████████████████████████████████████████| 4/4 [00:03<00:00,  1.05it/s]


Epoch : 2 - Train_loss : 1.3792 - Train_Acc: 0.3179 - Val_loss : 0.2717 - Val_Acc: 0.3000  - Time: 13.320768594741821



100%|█████████████████████████████████████████████| 4/4 [00:03<00:00,  1.19it/s]


...

Epoch : 498 - Train_loss : 0.9414 - Train_Acc: 0.6357 - Val_loss : 0.2116 - Val_Acc: 0.4083  - Time: 8.519229412078857



100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.54it/s]


Epoch : 499 - Train_loss : 0.9520 - Train_Acc: 0.6179 - Val_loss : 0.2116 - Val_Acc: 0.4083  - Time: 8.23508071899414



100%|█████████████████████████████████████████████| 4/4 [00:02<00:00,  1.54it/s]

Epoch : 500 - Train_loss : 0.9455 - Train_Acc: 0.6179 - Val_loss : 0.2119 - Val_Acc: 0.4083  - Time: 8.322201490402222

Finished Training


In [ ]:
from matplotlib import pyplot as plt
import numpy as np
epochs_range = range(500)#the x-axis of both the plots wil have the epoch ranges mentioned during each training specification.

# To plot validation accuracy versus each training epoch
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_accuracy, label='Training Accuracy')
plt.plot(epochs_range, np.array(val_accuracy), label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('No of Epochs')
plt.legend(loc='lower right')
plt.title('Training Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, np.array(train_loss), label='Training Loss')
plt.plot(epochs_range, np.array(val_loss), label='Validation Loss')
plt.ylabel('Loss')
plt.xlabel('No of Epochs')
plt.legend(loc='lower right')
plt.title('Validation Loss')